# Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [103]:
macro = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ДЗ. Большое 3/macro.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ДЗ. Большое 3/sample_submission.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ДЗ. Большое 3/test.csv')
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ДЗ. Большое 3/train.csv')

# Разбиение на числовые и категориальные признаки

In [104]:
pd.options.display.max_rows = 100

Общая информация о выборках.

In [105]:
# macro.info()

In [106]:
# train.info()

In [107]:
# test.info()

Разобьем выборки на числовые и категориальные признаки

In [108]:
train_num_cols = []
for i in train.columns:
  if (train[i].dtype == "int64") or (train[i].dtype == "float64"):
    train_num_cols.append(i)

test_num_cols = []
for i in test.columns:
  if (test[i].dtype == "int64") or (test[i].dtype == "float64"):
    test_num_cols.append(i)

macro_num_cols = []
for i in macro.columns:
  if (macro[i].dtype == "int64") or (macro[i].dtype == "float64"):
    macro_num_cols.append(i)

train_cat_cols = list(set(train.columns) - set(train_num_cols))
test_cat_cols = list(set(test.columns) - set(test_num_cols))
macro_cat_cols = list(set(macro.columns) - set(macro_num_cols))

In [109]:
# проверка
print(len(train_cat_cols) + len(train_num_cols) == len(list(train.columns)),
      len(test_cat_cols) + len(test_num_cols) == len(list(test.columns)),
      len(macro_cat_cols) + len(macro_num_cols) == len(list(macro.columns)))

True True True


# EDA. Числовые признаки / train, test выборки

## Матрица корреляций, удаление признаков

Построим матрицу корреляций числовых признаков: 

In [10]:
train_cor_num = train[train_num_cols].corr()

Найдем и удалим признаки с высокой корреляцией (> 0.8): 

In [11]:
# Поиск признаков с высокой корреляцией
correlated_features = set()

for i in range(len(train_cor_num.columns)): 
  for j in range(i): 
    if abs(train_cor_num.iloc[i, j]) > 0.8: 
      colname = train_cor_num.columns[i] 
      correlated_features.add(colname)

len(correlated_features)

186

In [12]:
# list(correlated_features)

In [13]:
# Удалим признаки с высокой корреляцией из train выборки
train.drop(labels=correlated_features, axis=1, inplace=True)
train_num_cols = list(set(train_num_cols)-set(correlated_features))
# Вычислим оставшееся колличество числовых признаков
len(train_num_cols)

90

In [110]:
# Удалим признаки с высокой корреляцией из test выборки
test.drop(labels=correlated_features, axis=1, inplace=True)
test_num_cols = list(set(test_num_cols)-set(correlated_features))
# Вычислим оставшееся колличество числовых признаков
len(test_num_cols)

89

In [15]:
train[train_num_cols]

,raion_popul,mosque_count_2000,school_education_centers_top_20_raion,market_shop_km,sport_count_1000,mosque_count_3000,big_market_km,ID_metro,church_synagogue_km,ID_railroad_terminal,railroad_station_walk_km,catering_km,university_top_20_raion,num_room,additional_education_raion,build_count_foam,trc_count_500,mosque_count_1500,mosque_count_5000,area_m,cafe_count_1000_price_high,industrial_km,life_sq,raion_build_count_with_material_info,build_count_1946-1970,water_km,ID_bus_terminal,trc_count_1000,cemetery_km,cafe_sum_5000_min_price_avg,full_sq,healthcare_centers_raion,cafe_sum_2000_min_price_avg,green_part_1000,market_count_1500,indust_part,build_count_block,mkad_km,green_part_500,sport_count_500,...,ID_railroad_station_walk,cafe_sum_1500_min_price_avg,big_road1_km,build_count_monolith,shopping_centers_raion,mosque_count_500,green_zone_km,fitness_km,build_count_wood,price_doc,big_road2_km,floor,mosque_count_1000,market_count_5000,culture_objects_top_25_raion,cafe_count_500_price_high,water_treatment_km,max_floor,metro_min_avto,green_zone_part,ice_rink_km,full_all,cafe_sum_3000_min_price_avg,kitch_sq,ID_big_road1,additional_education_km,ID_railroad_station_avto,bus_terminal_avto_km,incineration_km,cafe_sum_1000_min_price_avg,prom_part_500,mosque_km,trc_sqm_500,sport_objects_raion,office_sqm_500,prom_part_5000,id,build_count_brick,state,market_count_3000
0,155572,0,0,1.676258,6,0,10.814172,1,0.628187,101,5.419893,0.516838,0,NaN,3,0.0,0,0,1,6.407578e+06,0,1.080934,27.0,211.0,0.0,0.992631,1,3,1.804127,708.57,43,1,614.29,7.36,1,0.000070,25.0,1.422391,0.00,1,...,1.0,566.67,1.422391,2.0,16,0,0.600973,0.485841,0.0,5850000,3.830951,4.0,0,4,0,0,23.683460,NaN,2.590241,0.189727,1.107594,86206,639.68,NaN,1,0.947962,1,24.292406,3.633334,527.78,0.00,3.932040,0,7,0,13.31,1,0.0,NaN,1
1,115352,0,0,3.424716,2,0,6.910568,2,0.471447,32,3.411993,0.230287,0,NaN,1,0.0,0,0,1,9.589337e+06,0,0.966479,19.0,245.0,143.0,0.698081,2,5,4.655004,673.81,34,1,695.24,26.66,0,0.049637,83.0,9.503405,25.14,0,...,2.0,694.12,2.887377,4.0,3,0,0.065321,0.668364,1.0,6000000,3.103996,3.0,0,14,1,0,1.317476,NaN,0.936700,0.372602,8.972823,76284,631.03,NaN,2,1.072315,2,5.706113,8.648587,615.38,0.00,4.841544,0,6,0,27.47,2,67.0,NaN,1
2,101708,0,0,1.375443,5,0,5.752368,3,0.755946,5,1.277658,0.190462,0,NaN,1,0.0,0,0,0,4.808270e+06,0,0.939275,29.0,330.0,246.0,0.468265,3,0,3.381083,702.68,43,1,563.64,4.99,5,0.118537,59.0,5.604800,1.67,0,...,3.0,516.67,0.647250,4.0,0,0,0.453172,0.733101,0.0,5700000,2.927487,2.0,0,10,0,0,4.912660,NaN,2.120999,0.112560,1.978517,101982,697.44,NaN,3,0.391957,3,6.710302,11.996480,642.86,0.00,7.922152,0,5,0,21.58,3,206.0,NaN,6
3,178473,0,0,0.811275,3,0,27.892717,4,1.561505,83,4.291432,0.465820,0,NaN,6,0.0,0,0,0,1.258354e+07,0,0.451173,50.0,458.0,40.0,1.200336,1,6,2.017080,931.58,89,1,660.00,19.25,2,0.069753,9.0,2.677824,17.36,0,...,4.0,673.91,2.677824,50.0,11,0,0.106125,0.623484,51.0,13100000,2.780449,9.0,0,3,0,0,15.623710,NaN,1.489049,0.194703,6.483172,21155,718.75,NaN,1,0.892674,4,6.734618,14.317640,658.33,0.57,15.300449,0,17,0,3.89,4,124.0,NaN,3
4,108171,0,0,1.969738,7,1,9.155057,5,0.121681,113,0.853960,0.026102,2,NaN,2,0.0,1,0,2,8.398461e+06,1,0.392871,77.0,746.0,146.0,0.820294,4,3,2.936581,853.88,77,4,765.93,3.34,2,0.037316,48.0,11.616653,3.56,3,...,5.0,766.80,1.721834,16.0,10,0,0.236214,0.220288,0.0,16331452,3.133531,4.0,0,14,0,0,10.683540,NaN,1.257186,0.015234,3.975401,28179,853.03,NaN,4,0.810801,113,1.423428,11.903910,763.45,4.44,2.584370,45000,25,293699,10.92,5,643.0,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,175518,1,0,0.812253,3,1,8.820934,24,0.612359,101,3.147161,0.055377,0,2.0,3,0.0,2,0,1,1.005305e+07,0,0.702853,27.0,282.0,14.0,0.979218,4,3,1.933090,6

In [16]:
train_num_cols

['raion_popul',
 'mosque_count_2000',
 'school_education_centers_top_20_raion',
 'market_shop_km',
 'sport_count_1000',
 'mosque_count_3000',
 'big_market_km',
 'ID_metro',
 'church_synagogue_km',
 'ID_railroad_terminal',
 'railroad_station_walk_km',
 'catering_km',
 'university_top_20_raion',
 'num_room',
 'additional_education_raion',
 'build_count_foam',
 'trc_count_500',
 'mosque_count_1500',
 'mosque_count_5000',
 'area_m',
 'cafe_count_1000_price_high',
 'industrial_km',
 'life_sq',
 'raion_build_count_with_material_info',
 'build_count_1946-1970',
 'water_km',
 'ID_bus_terminal',
 'trc_count_1000',
 'cemetery_km',
 'cafe_sum_5000_min_price_avg',
 'full_sq',
 'healthcare_centers_raion',
 'cafe_sum_2000_min_price_avg',
 'green_part_1000',
 'market_count_1500',
 'indust_part',
 'build_count_block',
 'mkad_km',
 'green_part_500',
 'sport_count_500',
 'cafe_sum_500_min_price_avg',
 'build_year',
 'market_count_1000',
 'ID_big_road2',
 'prom_part_1000',
 'hospital_beds_raion',
 'marke

Определим колличество NaN в оставшихся признаках.

In [17]:
train[train_num_cols].isnull().sum().sort_values(ascending=False)

hospital_beds_raion                      14441
build_year                               13605
state                                    13559
cafe_sum_500_min_price_avg               13281
kitch_sq                                  9572
num_room                                  9572
material                                  9572
max_floor                                 9572
cafe_sum_1000_min_price_avg               6524
life_sq                                   6383
build_count_wood                          4991
build_count_monolith                      4991
build_count_frame                         4991
build_count_brick                         4991
build_count_1946-1970                     4991
build_count_foam                          4991
raion_build_count_with_material_info      4991
build_count_block                         4991
cafe_sum_1500_min_price_avg               4199
cafe_sum_2000_min_price_avg               1725
cafe_sum_3000_min_price_avg                991
cafe_sum_5000

Удалим признаки с NaN, апроксимация которых не целесообразна (не привнесет полезных данных, внесет только шум), а также признаки, не несущие большой значимости. 

In [18]:
# список признаков для удаления 
del_num_cols = []

# удалим признаки c большим колличеством NaN, апроксимация которых не целесообразна: 
for i in ['state', 'material', 'kitch_sq', 'max_floor', 'num_room', 'life_sq']:
  del_num_cols.append(i)

# удалим признаки, не несущие большой значимости: 
for i in ['build_count_block', 'raion_build_count_with_material_info', 
                     'build_count_brick', 'build_count_foam', 'build_count_frame', 'build_count_monolith']:
  del_num_cols.append(i)

In [19]:
train.drop(labels=del_num_cols, axis=1, inplace=True)
train_num_cols = list(set(train_num_cols)-set(del_num_cols))
len(train_num_cols)

78

In [111]:
test.drop(labels=del_num_cols, axis=1, inplace=True)
test_num_cols = list(set(test_num_cols)-set(del_num_cols))
len(test_num_cols)

77

## Замена NaN

В оставшихся признаках заменим NaN используя следующую логику. В окрестностях одной станции метро (железнодорожного терминала) квартиры будут имеют примерно одиноковые признаки: год постройки здания, колличество больниц в округе, средний чек в близлежащих ресторанах, колличество промышленных территорий, колличество этажей в зданиях и т.д. NaN по каждому признаку будем заменять модой среди значений с однаковым значением ID_metro (ID_railroad_terminal).  

In [21]:
# функция для нахождения моды искомого признака по целевому признаку
def mean_fuature (data,feature,target_feature):
  return data[[target_feature,feature]].groupby(target_feature, dropna=True).apply(lambda x: x.mode().iloc[0])

In [22]:
# следующие NaN заменим наиболее частым значением (мода) среди сэмплов с аналогичным значением 'ID_metro'.
replace_num_col = ['hospital_beds_raion', 'build_year', 'cafe_sum_500_min_price_avg', 'cafe_sum_1000_min_price_avg', 
                         'build_count_wood', 'build_count_1946-1970', 'cafe_sum_1500_min_price_avg', 'cafe_sum_2000_min_price_avg', 
                         'cafe_sum_3000_min_price_avg', 'cafe_sum_5000_min_price_avg', 'prom_part_5000','floor', 'railroad_station_walk_km',
                         'ID_railroad_station_walk']

In [23]:
# удостоверимся, что для всех значений 'ID_metro' есть хотябы одно значение отличное от NaN для каждого из признаков train_replace_num_col
for i in replace_num_col:
  check_nan = mean_fuature(train,i,'ID_metro').notnull()
  print(i,'/',len(check_nan[check_nan[i]==False][i]),'/',list(check_nan[check_nan[i]==False].index))

hospital_beds_raion / 33 / [4, 6, 11, 16, 18, 19, 23, 36, 37, 45, 60, 62, 73, 74, 80, 101, 102, 104, 127, 135, 142, 151, 154, 157, 172, 179, 183, 198, 201, 214, 219, 222, 223]
build_year / 4 / [159, 160, 171, 198]
cafe_sum_500_min_price_avg / 6 / [108, 155, 171, 185, 188, 223]
cafe_sum_1000_min_price_avg / 1 / [108]
build_count_wood / 0 / []
build_count_1946-1970 / 0 / []
cafe_sum_1500_min_price_avg / 0 / []
cafe_sum_2000_min_price_avg / 0 / []
cafe_sum_3000_min_price_avg / 0 / []
cafe_sum_5000_min_price_avg / 0 / []
prom_part_5000 / 0 / []
floor / 0 / []
railroad_station_walk_km / 0 / []
ID_railroad_station_walk / 0 / []


Как видно, признаки hospital_beds_raion, build_year, cafe_sum_500_min_price_avg, cafe_sum_1000_min_price_avg не позволяют сделать полную аппроксимацию по моде 'ID_metro'.

Удостоверимся, можно ли вычислить их NaN с помощью моды ID_railroad_terminal.
Итог - можно.

In [24]:
for i in ['hospital_beds_raion', 'build_year', 'cafe_sum_500_min_price_avg', 'cafe_sum_1000_min_price_avg']:
  check_nan = mean_fuature(train,i,'ID_railroad_terminal').notnull()
  print(i,'/',len(check_nan[check_nan[i]==False][i]),'/',list(check_nan[check_nan[i]==False].index))

hospital_beds_raion / 0 / []
build_year / 0 / []
cafe_sum_500_min_price_avg / 0 / []
cafe_sum_1000_min_price_avg / 0 / []


In [112]:
# скопируем датафреймы  
train_copy = train.copy()
test_copy = test.copy()

Разобьем список признаков, содержащих NaN, на две части (по целевым признакам).

In [26]:
import pickle

In [27]:
# сформируем словарь для замены 
def values(tagret_fuature, counter, list):
  return {i:mean_fuature(train_copy,i,tagret_fuature).at[counter,i] for i in list}

1) для ID_metro:

In [28]:
# признаки для ID_metro
f1 = ['build_count_wood', 'build_count_1946-1970', 'cafe_sum_1500_min_price_avg', 'cafe_sum_2000_min_price_avg', 
      'cafe_sum_3000_min_price_avg', 'cafe_sum_5000_min_price_avg', 'prom_part_5000','floor', 'railroad_station_walk_km',
      'ID_railroad_station_walk']

In [29]:
# уникальные ID_metro
ID_metro_st = list(set(train['ID_metro'].to_list()))

In [30]:
# замена NaN
%%time
for j in ID_metro_st:
  train_without_nan = train_copy.fillna(value = values('ID_metro', j, f1))

CPU times: user 12min 15s, sys: 17.6 s, total: 12min 32s
Wall time: 12min 7s


In [113]:
%%time
for j in ID_metro_st:
  test_without_nan = test_copy.fillna(value = values('ID_metro', j, f1))

CPU times: user 12min 18s, sys: 18.4 s, total: 12min 36s
Wall time: 12min 9s


2) для ID_railroad_terminal:

In [32]:
# признаки для ID_railroad_terminal
f2 = ['hospital_beds_raion', 'build_year', 'cafe_sum_500_min_price_avg', 'cafe_sum_1000_min_price_avg']

In [33]:
# уникальные ID_railroad_terminal
ID_rail_term = list(set(train['ID_railroad_terminal'].to_list()))

In [34]:
# замена NaN
for j in ID_rail_term:
  train_without_nan = train_without_nan.fillna(value = values('ID_railroad_terminal', j, f2))

In [114]:
for j in ID_rail_term:
  test_without_nan = test_without_nan.fillna(value = values('ID_railroad_terminal', j, f2))

Удалим из train искомый признак 'price_doc' и сохраним выборку.

In [36]:
train_without_nan = train_without_nan.drop(['price_doc'], axis=1)

In [37]:
save_train_without_nan = pickle.dumps(train_without_nan)

In [38]:
# train_without_nan = pickle.loads(save_train_without_nan)

In [39]:
train_without_nan

,id,timestamp,full_sq,floor,build_year,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,build_count_wood,build_count_1946-1970,ID_metro,metro_min_avto,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,ID_railroad_station_walk,...,big_market_km,market_shop_km,fitness_km,ice_rink_km,additional_education_km,church_synagogue_km,mosque_km,catering_km,ecology,green_part_500,prom_part_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_sum_500_min_price_avg,cafe_count_500_price_high,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,trc_count_1000,cafe_sum_1000_min_price_avg,cafe_count_1000_price_high,mosque_count_1000,sport_count_1000,market_count_1000,cafe_sum_1500_min_price_avg,mosque_count_1500,market_count_1500,cafe_sum_2000_min_price_avg,mosque_count_2000,cafe_sum_3000_min_price_avg,mosque_count_3000,market_count_3000,prom_part_5000,cafe_sum_5000_min_price_avg,mosque_count_5000,market_count_5000
0,1,2011-08-20,43,4.0,2014.0,Investment,Bibirevo,6.407578e+06,155572,0.189727,0.000070,0,240.0,1,0,7,3,no,0,16,no,no,no,no,no,no,no,no,86206,0.0,0.0,1,2.590241,0.600973,1.080934,23.683460,1.804127,3.633334,5.419893,1.0,...,10.814172,1.676258,0.485841,1.107594,0.947962,0.628187,3.932040,0.516838,good,0.00,0.00,0,0,0,1000.00,0,0,0,1,0,7.36,0.00,3,527.78,0,0,6,1,566.67,0,1,614.29,0,639.68,0,1,13.31,708.57,1,4
1,2,2011-08-23,34,3.0,2014.0,Investment,Nagatinskij Zaton,9.589337e+06,115352,0.372602,0.049637,0,229.0,1,0,6,1,yes,1,3,no,no,no,no,no,no,no,no,76284,1.0,143.0,2,0.936700,0.065321,0.966479,1.317476,4.655004,8.648587,3.411993,2.0,...,6.910568,3.424716,0.668364,8.972823,1.072315,0.471447,4.841544,0.230287,excellent,25.14,0.00,0,0,0,860.00,0,0,0,0,0,26.66,0.07,5,615.38,0,0,2,0,694.12,0,0,695.24,0,631.03,0,1,27.47,673.81,1,14
2,3,2011-08-27,43,2.0,2014.0,Investment,Tekstil'shhiki,4.808270e+06,101708,0.112560,0.118537,0,1183.0,1,0,5,1,no,0,0,no,no,no,yes,no,no,no,no,101982,0.0,246.0,3,2.120999,0.453172,0.939275,4.912660,3.381083,11.996480,1.277658,3.0,...,5.752368,1.375443,0.733101,1.978517,0.391957,0.755946,7.922152,0.190462,poor,1.67,0.00,0,0,0,666.67,0,0,0,0,0,4.99,0.29,0,642.86,0,0,5,3,516.67,0,5,563.64,0,697.44,0,6,21.58,702.68,0,10
3,4,2011-09-01,89,9.0,2014.0,Investment,Mitino,1.258354e+07,178473,0.194703,0.069753,0,1046.0,1,0,17,6,no,0,11,no,no,no,no,no,no,no,no,21155,51.0,40.0,4,1.489049,0.106125,0.451173,15.623710,2.017080,14.317640,4.291432,4.0,...,27.892717,0.811275,0.623484,6.483172,0.892674,1.561505,15.300449,0.465820,good,17.36,0.57,0,0,0,1000.00,0,0,0,0,0,19.25,10.35,6,658.33,0,0,3,1,673.91,0,2,660.00,0,718.75,0,3,3.89,931.58,0,3
4,5,2011-09-05,77,4.0,2014.0,Investment,Basmannoe,8.398461e+06,108171,0.015234,0.037316,0,562.0,4,2,25,2,no,0,10,no,no,no,yes,yes,no,no,no,28179,0.0,146.0,5,1.257186,0.236214,0.392871,10.683540,2.936581,11.903910,0.853960,5.0,...,9.155057,1.969738,0.220288,3.975401,0.810801,0.121681,2.584370,0.026102,excellent,3.56,4.44,293699,1,45000,702.22,0,0,2,3,0,3.34,8.29,3,763.45,1,0,7,0,766.80,0,2,765.93,0,853.03,1,5,10.92,853.88,2,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,30469,2015-06-30,44,7.0,1975.0,Investment,Otradnoe,1.005305e+07,175518,0.096270,0.300323,0,1046.0,1,0,10,3,no,0,9,no,yes,no,yes,no,no,

In [115]:
test_without_nan

,id,timestamp,full_sq,floor,build_year,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,build_count_wood,build_count_1946-1970,ID_metro,metro_min_avto,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,ID_railroad_station_walk,...,big_market_km,market_shop_km,fitness_km,ice_rink_km,additional_education_km,church_synagogue_km,mosque_km,catering_km,ecology,green_part_500,prom_part_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_sum_500_min_price_avg,cafe_count_500_price_high,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,trc_count_1000,cafe_sum_1000_min_price_avg,cafe_count_1000_price_high,mosque_count_1000,sport_count_1000,market_count_1000,cafe_sum_1500_min_price_avg,mosque_count_1500,market_count_1500,cafe_sum_2000_min_price_avg,mosque_count_2000,cafe_sum_3000_min_price_avg,mosque_count_3000,market_count_3000,prom_part_5000,cafe_sum_5000_min_price_avg,mosque_count_5000,market_count_5000
0,30474,2015-07-01,39.00,2,1998.0,Investment,Juzhnoe Butovo,2.615514e+07,178264,0.137846,0.041116,1,1046.0,1,0,13,4,no,0,4,no,no,no,no,no,no,no,no,102618,607.0,439.0,45,1.258957,0.061485,1.205404,0.967696,0.781053,10.565400,4.812102,39.0,...,15.483912,5.353674,0.225788,11.810839,0.514211,0.848297,1.917736,0.446324,satisfactory,42.22,0.00,0,0,0,1000.00,0,0,0,1,0,32.48,0.00,0,1000.00,0,0,3,0,1000.00,0,0,1000.00,1,781.82,1,0,4.69,676.47,1,1
1,30475,2015-07-01,79.20,8,0.0,OwnerOccupier,Poselenie Vnukovskoe,2.553630e+07,4001,0.496315,0.007122,0,1046.0,0,0,0,0,no,0,1,no,no,no,no,no,no,no,no,17790,3.0,281.0,21,4.230425,0.000000,0.742377,16.049420,2.244906,18.500540,5.458057,24.0,...,15.092542,8.156185,1.313180,4.438242,1.694967,1.162371,12.239901,0.705873,no data,86.33,0.00,0,0,0,1000.00,0,0,0,0,0,67.43,3.77,0,1000.00,0,0,0,0,833.33,0,0,757.14,0,680.00,0,0,7.70,733.33,0,1
2,30476,2015-07-01,40.50,3,1960.0,Investment,Perovo,9.946335e+06,139322,0.065409,0.225825,0,1548.0,3,0,13,0,no,0,2,yes,no,yes,yes,no,no,no,no,36154,0.0,437.0,44,1.585306,0.580638,0.900408,11.749900,3.389848,10.195630,3.628293,68.0,...,16.304596,0.644830,0.966254,3.131143,2.330995,1.177527,9.938735,0.320864,poor,0.00,0.00,0,0,0,400.00,0,0,0,0,1,13.07,0.80,0,485.71,0,0,6,2,655.56,0,3,633.33,0,608.33,0,4,13.59,668.97,0,11
3,30477,2015-07-01,62.80,17,2016.0,OwnerOccupier,Poselenie Voskresenskoe,2.149409e+07,7122,0.262459,0.017647,0,1046.0,0,0,0,2,no,0,0,no,no,no,no,no,no,no,no,9553,3.0,281.0,45,7.931398,0.025446,0.466738,5.061917,2.701804,14.629440,10.284167,39.0,...,10.723870,11.112624,4.480234,15.200509,0.403429,0.213853,6.153091,0.454087,no data,22.01,0.15,0,0,0,300.00,0,0,0,0,0,27.94,7.90,0,300.00,0,0,0,0,300.00,0,0,300.00,0,300.00,0,0,1.66,1560.00,0,0
4,30478,2015-07-01,40.00,17,0.0,OwnerOccupier,Poselenie Vnukovskoe,2.553630e+07,4001,0.496315,0.007122,0,1046.0,0,0,0,0,no,0,1,no,no,no,no,no,no,no,no,17790,3.0,281.0,21,2.152792,0.427248,0.353642,16.784630,2.250137,19.149190,3.735666,24.0,...,15.546028,6.433794,1.519553,2.715850,1.339078,1.186621,12.652956,0.066503,no data,3.33,3.70,0,0,0,1000.00,0,0,0,0,0,36.66,2.92,0,1000.00,0,0,0,0,833.33,0,0,757.14,0,700.00,0,0,6.96,747.37,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,38131,2016-05-26,52.20,10,1973.0,Investment,Kon'kovo,7.81

# EDA. Числовые признаки / macro выборка

In [41]:
macro_cor_num = macro[macro_num_cols].corr()

In [42]:
# Поиск признаков с высокой корреляцией
macro_correlated_features = set()

for i in range(len(macro_cor_num.columns)): 
  for j in range(i): 
    if abs(macro_cor_num.iloc[i, j]) > 0.8: 
      colname = macro_cor_num.columns[i] 
      macro_correlated_features.add(colname)

len(macro_correlated_features)

74

In [43]:
# Удалим признаки с высокой корреляцией 
macro.drop(labels=macro_correlated_features, axis=1, inplace=True)
macro_num_cols = list(set(macro_num_cols)-set(macro_correlated_features))
# Вычислим оставшееся колличество числовых признаков
len(macro_num_cols)

22

In [44]:
macro[macro_num_cols]

,real_dispos_income_per_cap_growth,rent_price_1room_eco,unemployment,mortgage_value,mortgage_rate,average_provision_of_build_contract,micex,balance_trade,deposits_growth,micex_rgbi_tr,overdue_wages_per_cap,gdp_quart_growth,balance_trade_growth,oil_urals,rent_price_2room_eco,income_per_cap,rent_price_4+room_bus,marriages_per_1000_cap,average_provision_of_build_contract_moscow,deposits_rate,net_capital_export,gdp_quart
0,0.038,NaN,0.018,142968,13.72,5.0,NaN,NaN,NaN,NaN,12865.0,NaN,NaN,76.1000,NaN,30789.2,NaN,8.0,NaN,NaN,NaN,NaN
1,0.038,NaN,0.018,142968,13.72,5.0,NaN,NaN,NaN,NaN,12865.0,NaN,NaN,76.1000,NaN,30789.2,NaN,8.0,NaN,NaN,NaN,NaN
2,0.038,NaN,0.018,142968,13.72,5.0,NaN,NaN,NaN,NaN,12865.0,NaN,NaN,76.1000,NaN,30789.2,NaN,8.0,NaN,NaN,NaN,NaN
3,0.038,NaN,0.018,142968,13.72,5.0,NaN,NaN,NaN,NaN,12865.0,NaN,NaN,76.1000,NaN,30789.2,NaN,8.0,NaN,NaN,NaN,NaN
4,0.038,NaN,0.018,142968,13.72,5.0,NaN,NaN,NaN,NaN,12865.0,NaN,NaN,76.1000,NaN,30789.2,NaN,8.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,NaN,29.78,NaN,899539,12.66,6.2,1966.39,5.823,0.016931,133.55,NaN,-0.6,2.6,44.3677,38.4,NaN,180.76,NaN,7.71,NaN,-0.408111,19979.4
2480,NaN,29.78,NaN,899539,12.66,6.2,1966.39,5.823,0.016931,133.55,NaN,-0.6,2.6,44.3677,38.4,NaN,180.76,NaN,7.71,NaN,-0.408111,19979.4
2481,NaN,29.78,NaN,899539,12.66,6.2,1956.95,5.823,0.016931,133.09,NaN,-0.6,2.6,44.3677,38.4,NaN,180.76,NaN,7.71,NaN,-0.408111,19979.4
2482,NaN,29.78,NaN,899539,12.66,6.2,1969.11,5.823,0.016931,132.88,NaN,-0.6,2.6,44.3677,38.4,NaN,180.76,NaN,7.71,NaN,-0.408111,19979.4


In [45]:
macro_num_cols

['real_dispos_income_per_cap_growth',
 'rent_price_1room_eco',
 'unemployment',
 'mortgage_value',
 'mortgage_rate',
 'average_provision_of_build_contract',
 'micex',
 'balance_trade',
 'deposits_growth',
 'micex_rgbi_tr',
 'overdue_wages_per_cap',
 'gdp_quart_growth',
 'balance_trade_growth',
 'oil_urals',
 'rent_price_2room_eco',
 'income_per_cap',
 'rent_price_4+room_bus',
 'marriages_per_1000_cap',
 'average_provision_of_build_contract_moscow',
 'deposits_rate',
 'net_capital_export',
 'gdp_quart']

In [46]:
# найдем колличество NaN по каждому числовому признаку
macro[macro_num_cols].isnull().sum(axis=0).sort_values(ascending=False).head(90)

real_dispos_income_per_cap_growth             658
marriages_per_1000_cap                        658
overdue_wages_per_cap                         658
deposits_rate                                 414
net_capital_export                            396
average_provision_of_build_contract_moscow    365
unemployment                                  293
income_per_cap                                293
rent_price_4+room_bus                         273
rent_price_2room_eco                          273
rent_price_1room_eco                          273
balance_trade_growth                           90
gdp_quart_growth                               90
gdp_quart                                      90
deposits_growth                                31
balance_trade                                  31
micex_rgbi_tr                                  10
micex                                          10
oil_urals                                       0
average_provision_of_build_contract             0


In [47]:
macro_copy = macro.copy()

Апроксимацию NaN в macro выборке будет производить при помощи встроенных методов.

In [48]:
macro_copy[macro_num_cols] = macro_copy[macro_num_cols].interpolate(method='linear', axis=0).fillna(method='bfill')

In [49]:
macro_without_nan = macro_copy

# Категориальные признаки / train, test, macro

In [50]:
from sklearn import preprocessing
ordinalencoder = preprocessing.OrdinalEncoder() 

In [51]:
train_without_nan[train_cat_cols]

,incineration_raion,culture_objects_top_25,timestamp,big_road1_1line,water_1line,sub_area,product_type,thermal_power_plant_raion,railroad_1line,railroad_terminal_raion,oil_chemistry_raion,nuclear_reactor_raion,big_market_raion,detention_facility_raion,ecology,radiation_raion
0,no,no,2011-08-20,no,no,Bibirevo,Investment,no,no,no,no,no,no,no,good,no
1,no,yes,2011-08-23,no,no,Nagatinskij Zaton,Investment,no,no,no,no,no,no,no,excellent,no
2,no,no,2011-08-27,no,no,Tekstil'shhiki,Investment,no,no,no,no,no,no,no,poor,yes
3,no,no,2011-09-01,no,no,Mitino,Investment,no,no,no,no,no,no,no,good,no
4,no,no,2011-09-05,no,no,Basmannoe,Investment,no,yes,yes,no,no,no,no,excellent,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,yes,no,2015-06-30,no,no,Otradnoe,Investment,no,no,no,no,no,no,no,good,yes
30467,no,yes,2015-06-30,no,no,Tverskoe,Investment,no,no,yes,no,no,no,yes,poor,yes
30468,no,no,2015-06-30,no,no,Poselenie Vnukovskoe,OwnerOccupier,no,no,no,no,no,no,no,no data,no
30469,no,no,2015-06-30,no,no,Obruchevskoe,Investment,no,no,no,no,no,no,no,satisfactory,yes


In [52]:
train_cat_cols

['incineration_raion',
 'culture_objects_top_25',
 'timestamp',
 'big_road1_1line',
 'water_1line',
 'sub_area',
 'product_type',
 'thermal_power_plant_raion',
 'railroad_1line',
 'railroad_terminal_raion',
 'oil_chemistry_raion',
 'nuclear_reactor_raion',
 'big_market_raion',
 'detention_facility_raion',
 'ecology',
 'radiation_raion']

Проверим выборку на наличие NaN. Первоначально, забыл проверить категориальные признаки на наличие NaN. Ошибка всплыла уже на этапе ML.

In [53]:
train_without_nan[train_cat_cols].isnull().sum().sort_values(ascending=False)

incineration_raion           0
culture_objects_top_25       0
timestamp                    0
big_road1_1line              0
water_1line                  0
sub_area                     0
product_type                 0
thermal_power_plant_raion    0
railroad_1line               0
railroad_terminal_raion      0
oil_chemistry_raion          0
nuclear_reactor_raion        0
big_market_raion             0
detention_facility_raion     0
ecology                      0
radiation_raion              0
dtype: int64

Для train все ок. А вот в test нашлись NaN. 

In [116]:
test_without_nan[test_cat_cols].isnull().sum().sort_values(ascending=False)

product_type                 33
incineration_raion            0
culture_objects_top_25        0
timestamp                     0
big_road1_1line               0
water_1line                   0
sub_area                      0
thermal_power_plant_raion     0
railroad_1line                0
railroad_terminal_raion       0
oil_chemistry_raion           0
nuclear_reactor_raion         0
big_market_raion              0
detention_facility_raion      0
ecology                       0
radiation_raion               0
dtype: int64

Заменим их используя втроенный метод инерполяции.

In [117]:
test_without_nan['product_type'].interpolate(method='linear', axis=0, inplace=True)

In [119]:
test_without_nan = test_without_nan.fillna(value = 'product_type')

In [120]:
test_without_nan

,id,timestamp,full_sq,floor,build_year,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,build_count_wood,build_count_1946-1970,ID_metro,metro_min_avto,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,ID_railroad_station_walk,...,big_market_km,market_shop_km,fitness_km,ice_rink_km,additional_education_km,church_synagogue_km,mosque_km,catering_km,ecology,green_part_500,prom_part_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_sum_500_min_price_avg,cafe_count_500_price_high,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,trc_count_1000,cafe_sum_1000_min_price_avg,cafe_count_1000_price_high,mosque_count_1000,sport_count_1000,market_count_1000,cafe_sum_1500_min_price_avg,mosque_count_1500,market_count_1500,cafe_sum_2000_min_price_avg,mosque_count_2000,cafe_sum_3000_min_price_avg,mosque_count_3000,market_count_3000,prom_part_5000,cafe_sum_5000_min_price_avg,mosque_count_5000,market_count_5000
0,30474,2015-07-01,39.00,2,1998.0,Investment,Juzhnoe Butovo,2.615514e+07,178264,0.137846,0.041116,1,1046.0,1,0,13,4,no,0,4,no,no,no,no,no,no,no,no,102618,607.0,439.0,45,1.258957,0.061485,1.205404,0.967696,0.781053,10.565400,4.812102,39.0,...,15.483912,5.353674,0.225788,11.810839,0.514211,0.848297,1.917736,0.446324,satisfactory,42.22,0.00,0,0,0,1000.00,0,0,0,1,0,32.48,0.00,0,1000.00,0,0,3,0,1000.00,0,0,1000.00,1,781.82,1,0,4.69,676.47,1,1
1,30475,2015-07-01,79.20,8,0.0,OwnerOccupier,Poselenie Vnukovskoe,2.553630e+07,4001,0.496315,0.007122,0,1046.0,0,0,0,0,no,0,1,no,no,no,no,no,no,no,no,17790,3.0,281.0,21,4.230425,0.000000,0.742377,16.049420,2.244906,18.500540,5.458057,24.0,...,15.092542,8.156185,1.313180,4.438242,1.694967,1.162371,12.239901,0.705873,no data,86.33,0.00,0,0,0,1000.00,0,0,0,0,0,67.43,3.77,0,1000.00,0,0,0,0,833.33,0,0,757.14,0,680.00,0,0,7.70,733.33,0,1
2,30476,2015-07-01,40.50,3,1960.0,Investment,Perovo,9.946335e+06,139322,0.065409,0.225825,0,1548.0,3,0,13,0,no,0,2,yes,no,yes,yes,no,no,no,no,36154,0.0,437.0,44,1.585306,0.580638,0.900408,11.749900,3.389848,10.195630,3.628293,68.0,...,16.304596,0.644830,0.966254,3.131143,2.330995,1.177527,9.938735,0.320864,poor,0.00,0.00,0,0,0,400.00,0,0,0,0,1,13.07,0.80,0,485.71,0,0,6,2,655.56,0,3,633.33,0,608.33,0,4,13.59,668.97,0,11
3,30477,2015-07-01,62.80,17,2016.0,OwnerOccupier,Poselenie Voskresenskoe,2.149409e+07,7122,0.262459,0.017647,0,1046.0,0,0,0,2,no,0,0,no,no,no,no,no,no,no,no,9553,3.0,281.0,45,7.931398,0.025446,0.466738,5.061917,2.701804,14.629440,10.284167,39.0,...,10.723870,11.112624,4.480234,15.200509,0.403429,0.213853,6.153091,0.454087,no data,22.01,0.15,0,0,0,300.00,0,0,0,0,0,27.94,7.90,0,300.00,0,0,0,0,300.00,0,0,300.00,0,300.00,0,0,1.66,1560.00,0,0
4,30478,2015-07-01,40.00,17,0.0,OwnerOccupier,Poselenie Vnukovskoe,2.553630e+07,4001,0.496315,0.007122,0,1046.0,0,0,0,0,no,0,1,no,no,no,no,no,no,no,no,17790,3.0,281.0,21,2.152792,0.427248,0.353642,16.784630,2.250137,19.149190,3.735666,24.0,...,15.546028,6.433794,1.519553,2.715850,1.339078,1.186621,12.652956,0.066503,no data,3.33,3.70,0,0,0,1000.00,0,0,0,0,0,36.66,2.92,0,1000.00,0,0,0,0,833.33,0,0,757.14,0,700.00,0,0,6.96,747.37,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,38131,2016-05-26,52.20,10,1973.0,Investment,Kon'kovo,7.81

In [121]:
test_without_nan[test_cat_cols].isnull().sum().sort_values(ascending=False)

incineration_raion           0
culture_objects_top_25       0
timestamp                    0
big_road1_1line              0
water_1line                  0
sub_area                     0
product_type                 0
thermal_power_plant_raion    0
railroad_1line               0
railroad_terminal_raion      0
oil_chemistry_raion          0
nuclear_reactor_raion        0
big_market_raion             0
detention_facility_raion     0
ecology                      0
radiation_raion              0
dtype: int64

Проведем анализ категориальных признаков. Как видим, большая часть признаков носит бинарный характер. Закодируем признаки с помощью OrdinalEncoder.

In [57]:
for feature in train_cat_cols:
    print(f'Feature {feature}')
    display(train_without_nan[feature].value_counts())
    print('-' * 40)

Feature incineration_raion


no     28155
yes     2316
Name: incineration_raion, dtype: int64

----------------------------------------
Feature culture_objects_top_25


no     28543
yes     1928
Name: culture_objects_top_25, dtype: int64

----------------------------------------
Feature timestamp


...

----------------------------------------
Feature ecology


poor            8018
no data         7656
good            7174
excellent       3938
satisfactory    3685
Name: ecology, dtype: int64

----------------------------------------
Feature radiation_raion


no     19600
yes    10871
Name: radiation_raion, dtype: int64

----------------------------------------


In [58]:
train_cat_cols.remove('timestamp')

In [59]:
# закодируем признаки
train_cat_code_feach = pd.DataFrame(ordinalencoder.fit_transform(train_without_nan[train_cat_cols]), columns=train_cat_cols)
train_cat_code_feach

,incineration_raion,culture_objects_top_25,big_road1_1line,water_1line,sub_area,product_type,thermal_power_plant_raion,railroad_1line,railroad_terminal_raion,oil_chemistry_raion,nuclear_reactor_raion,big_market_raion,detention_facility_raion,ecology,radiation_raion
0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,129.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
3,0.0,0.0,0.0,0.0,65.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,6.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,1.0,0.0,0.0,0.0,82.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
30467,0.0,1.0,0.0,0.0,134.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,1.0
30468,0.0,0.0,0.0,0.0,103.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
30469,0.0,0.0,0.0,0.0,77.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0


In [122]:
test_cat_cols.remove('timestamp')

In [123]:
test_cat_code_feach = pd.DataFrame(ordinalencoder.fit_transform(test_without_nan[test_cat_cols]), columns=test_cat_cols)
test_cat_code_feach

,incineration_raion,culture_objects_top_25,big_road1_1line,water_1line,sub_area,product_type,thermal_power_plant_raion,railroad_1line,railroad_terminal_raion,oil_chemistry_raion,nuclear_reactor_raion,big_market_raion,detention_facility_raion,ecology,radiation_raion
0,0.0,0.0,0.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
1,0.0,0.0,0.0,0.0,102.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
2,0.0,0.0,0.0,0.0,84.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0
3,0.0,0.0,0.0,0.0,104.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
4,0.0,0.0,0.0,0.0,102.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,0.0,0.0,0.0,0.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0
7658,0.0,0.0,0.0,0.0,86.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
7659,0.0,1.0,0.0,0.0,133.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
7660,0.0,0.0,0.0,0.0,79.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0


Заменим категориальные признаки на закодированные. 

In [62]:
train_without_nan[train_cat_cols] = train_cat_code_feach

In [63]:
train_without_nan

,id,timestamp,full_sq,floor,build_year,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,build_count_wood,build_count_1946-1970,ID_metro,metro_min_avto,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,ID_railroad_station_walk,...,big_market_km,market_shop_km,fitness_km,ice_rink_km,additional_education_km,church_synagogue_km,mosque_km,catering_km,ecology,green_part_500,prom_part_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_sum_500_min_price_avg,cafe_count_500_price_high,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,trc_count_1000,cafe_sum_1000_min_price_avg,cafe_count_1000_price_high,mosque_count_1000,sport_count_1000,market_count_1000,cafe_sum_1500_min_price_avg,mosque_count_1500,market_count_1500,cafe_sum_2000_min_price_avg,mosque_count_2000,cafe_sum_3000_min_price_avg,mosque_count_3000,market_count_3000,prom_part_5000,cafe_sum_5000_min_price_avg,mosque_count_5000,market_count_5000
0,1,2011-08-20,43,4.0,2014.0,0.0,9.0,6.407578e+06,155572,0.189727,0.000070,0,240.0,1,0,7,3,0.0,0,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86206,0.0,0.0,1,2.590241,0.600973,1.080934,23.683460,1.804127,3.633334,5.419893,1.0,...,10.814172,1.676258,0.485841,1.107594,0.947962,0.628187,3.932040,0.516838,1.0,0.00,0.00,0,0,0,1000.00,0,0,0,1,0,7.36,0.00,3,527.78,0,0,6,1,566.67,0,1,614.29,0,639.68,0,1,13.31,708.57,1,4
1,2,2011-08-23,34,3.0,2014.0,0.0,70.0,9.589337e+06,115352,0.372602,0.049637,0,229.0,1,0,6,1,1.0,1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76284,1.0,143.0,2,0.936700,0.065321,0.966479,1.317476,4.655004,8.648587,3.411993,2.0,...,6.910568,3.424716,0.668364,8.972823,1.072315,0.471447,4.841544,0.230287,0.0,25.14,0.00,0,0,0,860.00,0,0,0,0,0,26.66,0.07,5,615.38,0,0,2,0,694.12,0,0,695.24,0,631.03,0,1,27.47,673.81,1,14
2,3,2011-08-27,43,2.0,2014.0,0.0,129.0,4.808270e+06,101708,0.112560,0.118537,0,1183.0,1,0,5,1,0.0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,101982,0.0,246.0,3,2.120999,0.453172,0.939275,4.912660,3.381083,11.996480,1.277658,3.0,...,5.752368,1.375443,0.733101,1.978517,0.391957,0.755946,7.922152,0.190462,3.0,1.67,0.00,0,0,0,666.67,0,0,0,0,0,4.99,0.29,0,642.86,0,0,5,3,516.67,0,5,563.64,0,697.44,0,6,21.58,702.68,0,10
3,4,2011-09-01,89,9.0,2014.0,0.0,65.0,1.258354e+07,178473,0.194703,0.069753,0,1046.0,1,0,17,6,0.0,0,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21155,51.0,40.0,4,1.489049,0.106125,0.451173,15.623710,2.017080,14.317640,4.291432,4.0,...,27.892717,0.811275,0.623484,6.483172,0.892674,1.561505,15.300449,0.465820,1.0,17.36,0.57,0,0,0,1000.00,0,0,0,0,0,19.25,10.35,6,658.33,0,0,3,1,673.91,0,2,660.00,0,718.75,0,3,3.89,931.58,0,3
4,5,2011-09-05,77,4.0,2014.0,0.0,6.0,8.398461e+06,108171,0.015234,0.037316,0,562.0,4,2,25,2,0.0,0,10,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,28179,0.0,146.0,5,1.257186,0.236214,0.392871,10.683540,2.936581,11.903910,0.853960,5.0,...,9.155057,1.969738,0.220288,3.975401,0.810801,0.121681,2.584370,0.026102,0.0,3.56,4.44,293699,1,45000,702.22,0,0,2,3,0,3.34,8.29,3,763.45,1,0,7,0,766.80,0,2,765.93,0,853.03,1,5,10.92,853.88,2,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,30469,2015-06-30,44,7.0,1975.0,0.0,82.0,1.005305e+07,175518,0.096270,0.300323,0,1046.0,1,0,10,3,0.0,0,9,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,61396,0.0,14.0,24,1.384021,0.139814,0.7028

In [124]:
test_without_nan[test_cat_cols] = test_cat_code_feach

In [125]:
test_without_nan

,id,timestamp,full_sq,floor,build_year,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,build_count_wood,build_count_1946-1970,ID_metro,metro_min_avto,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,ID_railroad_station_walk,...,big_market_km,market_shop_km,fitness_km,ice_rink_km,additional_education_km,church_synagogue_km,mosque_km,catering_km,ecology,green_part_500,prom_part_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_sum_500_min_price_avg,cafe_count_500_price_high,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,trc_count_1000,cafe_sum_1000_min_price_avg,cafe_count_1000_price_high,mosque_count_1000,sport_count_1000,market_count_1000,cafe_sum_1500_min_price_avg,mosque_count_1500,market_count_1500,cafe_sum_2000_min_price_avg,mosque_count_2000,cafe_sum_3000_min_price_avg,mosque_count_3000,market_count_3000,prom_part_5000,cafe_sum_5000_min_price_avg,mosque_count_5000,market_count_5000
0,30474,2015-07-01,39.00,2,1998.0,0.0,38.0,2.615514e+07,178264,0.137846,0.041116,1,1046.0,1,0,13,4,0.0,0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,102618,607.0,439.0,45,1.258957,0.061485,1.205404,0.967696,0.781053,10.565400,4.812102,39.0,...,15.483912,5.353674,0.225788,11.810839,0.514211,0.848297,1.917736,0.446324,4.0,42.22,0.00,0,0,0,1000.00,0,0,0,1,0,32.48,0.00,0,1000.00,0,0,3,0,1000.00,0,0,1000.00,1,781.82,1,0,4.69,676.47,1,1
1,30475,2015-07-01,79.20,8,0.0,1.0,102.0,2.553630e+07,4001,0.496315,0.007122,0,1046.0,0,0,0,0,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17790,3.0,281.0,21,4.230425,0.000000,0.742377,16.049420,2.244906,18.500540,5.458057,24.0,...,15.092542,8.156185,1.313180,4.438242,1.694967,1.162371,12.239901,0.705873,2.0,86.33,0.00,0,0,0,1000.00,0,0,0,0,0,67.43,3.77,0,1000.00,0,0,0,0,833.33,0,0,757.14,0,680.00,0,0,7.70,733.33,0,1
2,30476,2015-07-01,40.50,3,1960.0,0.0,84.0,9.946335e+06,139322,0.065409,0.225825,0,1548.0,3,0,13,0,0.0,0,2,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,36154,0.0,437.0,44,1.585306,0.580638,0.900408,11.749900,3.389848,10.195630,3.628293,68.0,...,16.304596,0.644830,0.966254,3.131143,2.330995,1.177527,9.938735,0.320864,3.0,0.00,0.00,0,0,0,400.00,0,0,0,0,1,13.07,0.80,0,485.71,0,0,6,2,655.56,0,3,633.33,0,608.33,0,4,13.59,668.97,0,11
3,30477,2015-07-01,62.80,17,2016.0,1.0,104.0,2.149409e+07,7122,0.262459,0.017647,0,1046.0,0,0,0,2,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9553,3.0,281.0,45,7.931398,0.025446,0.466738,5.061917,2.701804,14.629440,10.284167,39.0,...,10.723870,11.112624,4.480234,15.200509,0.403429,0.213853,6.153091,0.454087,2.0,22.01,0.15,0,0,0,300.00,0,0,0,0,0,27.94,7.90,0,300.00,0,0,0,0,300.00,0,0,300.00,0,300.00,0,0,1.66,1560.00,0,0
4,30478,2015-07-01,40.00,17,0.0,1.0,102.0,2.553630e+07,4001,0.496315,0.007122,0,1046.0,0,0,0,0,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17790,3.0,281.0,21,2.152792,0.427248,0.353642,16.784630,2.250137,19.149190,3.735666,24.0,...,15.546028,6.433794,1.519553,2.715850,1.339078,1.186621,12.652956,0.066503,2.0,3.33,3.70,0,0,0,1000.00,0,0,0,0,0,36.66,2.92,0,1000.00,0,0,0,0,833.33,0,0,757.14,0,700.00,0,0,6.96,747.37,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,38131,2016-05-26,52.20,10,1973.0,0.0,43.0,7.811375e+06,153248,0.127002,0.135650,0,350.0,3,0,11,3,0.0,0,11,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,123

Проведем аналогичные процедуры для macro выборки.

In [66]:
macro_cat_cols.remove('timestamp')
macro_cat_cols

['modern_education_share',
 'old_education_build_share',
 'child_on_acc_pre_school']

In [67]:
for feature in macro_cat_cols:
    print(f'Feature {feature}')
    display(macro_without_nan[feature].value_counts())
    print('-' * 40)

Feature modern_education_share


90,92      365
93,08      365
95,4918    365
Name: modern_education_share, dtype: int64

----------------------------------------
Feature old_education_build_share


23,14     365
25,47     365
8,2517    365
Name: old_education_build_share, dtype: int64

----------------------------------------
Feature child_on_acc_pre_school


7,311     366
45,713    365
#!        365
3,013     365
16,765    365
Name: child_on_acc_pre_school, dtype: int64

----------------------------------------


Категориальные macro признаки содержат большое колличество пропущенных значений. Аппроксимировать их не целесообразно. Удалим данные признаки.

In [68]:
macro_without_nan=macro_without_nan.drop(macro_cat_cols, axis=1)

In [69]:
macro_without_nan

,timestamp,oil_urals,gdp_quart,gdp_quart_growth,balance_trade,balance_trade_growth,net_capital_export,average_provision_of_build_contract,average_provision_of_build_contract_moscow,micex,micex_rgbi_tr,deposits_growth,deposits_rate,mortgage_value,mortgage_rate,income_per_cap,real_dispos_income_per_cap_growth,unemployment,overdue_wages_per_cap,marriages_per_1000_cap,rent_price_4+room_bus,rent_price_2room_eco,rent_price_1room_eco
0,2010-01-01,76.1000,9995.8,4.1,18.114,14.1,-0.136524,5.0,6.02,1444.78,130.32,0.002354,4.50,142968,13.72,30789.2,0.038,0.018000,12865.0,8.0,156.36,34.56,27.58
1,2010-01-02,76.1000,9995.8,4.1,18.114,14.1,-0.136524,5.0,6.02,1444.78,130.32,0.002354,4.50,142968,13.72,30789.2,0.038,0.018000,12865.0,8.0,156.36,34.56,27.58
2,2010-01-03,76.1000,9995.8,4.1,18.114,14.1,-0.136524,5.0,6.02,1444.78,130.32,0.002354,4.50,142968,13.72,30789.2,0.038,0.018000,12865.0,8.0,156.36,34.56,27.58
3,2010-01-04,76.1000,9995.8,4.1,18.114,14.1,-0.136524,5.0,6.02,1444.78,130.32,0.002354,4.50,142968,13.72,30789.2,0.038,0.018000,12865.0,8.0,156.36,34.56,27.58
4,2010-01-05,76.1000,9995.8,4.1,18.114,14.1,-0.136524,5.0,6.02,1444.78,130.32,0.002354,4.50,142968,13.72,30789.2,0.038,0.018000,12865.0,8.0,156.36,34.56,27.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,2016-10-15,44.3677,19979.4,-0.6,5.823,2.6,-0.408111,6.2,7.71,1966.39,133.55,0.016931,6.77,899539,12.66,102709.0,-0.079,0.017708,57746.0,8.3,180.76,38.40,29.78
2480,2016-10-16,44.3677,19979.4,-0.6,5.823,2.6,-0.408111,6.2,7.71,1966.39,133.55,0.016931,6.77,899539,12.66,102709.0,-0.079,0.017708,57746.0,8.3,180.76,38.40,29.78
2481,2016-10-17,44.3677,19979.4,-0.6,5.823,2.6,-0.408111,6.2,7.71,1956.95,133.09,0.016931,6.77,899539,12.66,102709.0,-0.079,0.017708,57746.0,8.3,180.76,38.40,29.78
2482,2016-10-18,44.3677,19979.4,-0.6,5.823,2.6,-0.408111,6.2,7.71,1969.11,132.88,0.016931,6.77,899539,12.66,102709.0,-0.079,0.017708,57746.0,8.3,180.76,38.40,29.78


# Обьединение (merge) train/test с macro

Объединим очищенные датафремы train_without_nan и macro_without_nan. Полученная выборка будет использоваться для обучения модели. 

In [70]:
train_final = pd.merge(train_without_nan, macro_without_nan, how='left', on='timestamp')

In [71]:
# удалим признак timestamp, так как он является категориальным и будет мешать нам при обучении моделей
train_final.drop('timestamp', axis=1, inplace=True)

In [72]:
train_final

,id,full_sq,floor,build_year,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,build_count_wood,build_count_1946-1970,ID_metro,metro_min_avto,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,ID_railroad_station_walk,ID_railroad_station_avto,...,trc_count_1000,cafe_sum_1000_min_price_avg,cafe_count_1000_price_high,mosque_count_1000,sport_count_1000,market_count_1000,cafe_sum_1500_min_price_avg,mosque_count_1500,market_count_1500,cafe_sum_2000_min_price_avg,mosque_count_2000,cafe_sum_3000_min_price_avg,mosque_count_3000,market_count_3000,prom_part_5000,cafe_sum_5000_min_price_avg,mosque_count_5000,market_count_5000,oil_urals,gdp_quart,gdp_quart_growth,balance_trade,balance_trade_growth,net_capital_export,average_provision_of_build_contract,average_provision_of_build_contract_moscow,micex,micex_rgbi_tr,deposits_growth,deposits_rate,mortgage_value,mortgage_rate,income_per_cap,real_dispos_income_per_cap_growth,unemployment,overdue_wages_per_cap,marriages_per_1000_cap,rent_price_4+room_bus,rent_price_2room_eco,rent_price_1room_eco
0,1,43,4.0,2014.0,0.0,9.0,6.407578e+06,155572,0.189727,0.000070,0,240.0,1,0,7,3,0.0,0,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86206,0.0,0.0,1,2.590241,0.600973,1.080934,23.683460,1.804127,3.633334,5.419893,1.0,1,...,3,527.78,0,0,6,1,566.67,0,1,614.29,0,639.68,0,1,13.31,708.57,1,4,109.3100,14313.7,3.3,15.459,10.1,0.301811,5.76,6.74,1438.74,131.16,0.009740,4.10,323275,11.84,42688.6,-0.005,0.014000,53636.0,8.5,136.11,36.77,29.07
1,2,34,3.0,2014.0,0.0,70.0,9.589337e+06,115352,0.372602,0.049637,0,229.0,1,0,6,1,1.0,1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76284,1.0,143.0,2,0.936700,0.065321,0.966479,1.317476,4.655004,8.648587,3.411993,2.0,2,...,5,615.38,0,0,2,0,694.12,0,0,695.24,0,631.03,0,1,27.47,673.81,1,14,109.3100,14313.7,3.3,15.459,10.1,0.301811,5.76,6.74,1444.11,131.45,0.009740,4.10,323275,11.84,42688.6,-0.005,0.014000,53636.0,8.5,136.11,36.77,29.07
2,3,43,2.0,2014.0,0.0,129.0,4.808270e+06,101708,0.112560,0.118537,0,1183.0,1,0,5,1,0.0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,101982,0.0,246.0,3,2.120999,0.453172,0.939275,4.912660,3.381083,11.996480,1.277658,3.0,3,...,0,642.86,0,0,5,3,516.67,0,5,563.64,0,697.44,0,6,21.58,702.68,0,10,109.3100,14313.7,3.3,15.459,10.1,0.301811,5.76,6.74,1458.84,131.08,0.009740,4.10,323275,11.84,42688.6,-0.005,0.014000,53636.0,8.5,136.11,36.77,29.07
3,4,89,9.0,2014.0,0.0,65.0,1.258354e+07,178473,0.194703,0.069753,0,1046.0,1,0,17,6,0.0,0,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21155,51.0,40.0,4,1.489049,0.106125,0.451173,15.623710,2.017080,14.317640,4.291432,4.0,4,...,6,658.33,0,0,3,1,673.91,0,2,660.00,0,718.75,0,3,3.89,931.58,0,3,111.2900,14313.7,3.3,15.386,10.1,0.599709,5.76,6.74,1551.58,131.45,0.009607,4.00,386388,11.92,40311.3,-0.005,0.014000,53636.0,8.5,155.22,37.73,30.63
4,5,77,4.0,2014.0,0.0,6.0,8.398461e+06,108171,0.015234,0.037316,0,562.0,4,2,25,2,0.0,0,10,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,28179,0.0,146.0,5,1.257186,0.236214,0.392871,10.683540,2.936581,11.903910,0.853960,5.0,113,...,3,763.45,1,0,7,0,766.80,0,2,765.93,0,853.03,1,5,10.92,853.88,2,14,111.2900,14313.7,3.3,15.386,10.1,0.599709,5.76,6.74,1492.83,131.19,0.018590,4.00,386388,11.92,40311.3,-0.005,0.014000,53636.0,8.5,155.22,37.73,30.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,304

In [126]:
test_final = pd.merge(test_without_nan, macro_without_nan, how='left', on='timestamp')

In [127]:
# удалим признак timestamp, так как он является категориальным и будет мешать нам при обучении моделей
test_final.drop('timestamp', axis=1,  inplace=True)

In [128]:
test_final

,id,full_sq,floor,build_year,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,build_count_wood,build_count_1946-1970,ID_metro,metro_min_avto,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,ID_railroad_station_walk,ID_railroad_station_avto,...,trc_count_1000,cafe_sum_1000_min_price_avg,cafe_count_1000_price_high,mosque_count_1000,sport_count_1000,market_count_1000,cafe_sum_1500_min_price_avg,mosque_count_1500,market_count_1500,cafe_sum_2000_min_price_avg,mosque_count_2000,cafe_sum_3000_min_price_avg,mosque_count_3000,market_count_3000,prom_part_5000,cafe_sum_5000_min_price_avg,mosque_count_5000,market_count_5000,oil_urals,gdp_quart,gdp_quart_growth,balance_trade,balance_trade_growth,net_capital_export,average_provision_of_build_contract,average_provision_of_build_contract_moscow,micex,micex_rgbi_tr,deposits_growth,deposits_rate,mortgage_value,mortgage_rate,income_per_cap,real_dispos_income_per_cap_growth,unemployment,overdue_wages_per_cap,marriages_per_1000_cap,rent_price_4+room_bus,rent_price_2room_eco,rent_price_1room_eco
0,30474,39.00,2,1998.0,0.0,38.0,2.615514e+07,178264,0.137846,0.041116,1,1046.0,1,0,13,4,0.0,0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,102618,607.0,439.0,45,1.258957,0.061485,1.205404,0.967696,0.781053,10.565400,4.812102,39.0,39,...,0,1000.00,0,0,3,0,1000.00,0,0,1000.00,1,781.82,1,0,4.69,676.47,1,1,55.8961,19284.1,-4.5,14.788,18.6,-0.065116,5.97,7.12,1639.70,118.73,0.026267,8.74,458999,13.29,63194.0,-0.079,0.017708,57746.0,8.3,211.83,39.62,30.98
1,30475,79.20,8,0.0,1.0,102.0,2.553630e+07,4001,0.496315,0.007122,0,1046.0,0,0,0,0,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17790,3.0,281.0,21,4.230425,0.000000,0.742377,16.049420,2.244906,18.500540,5.458057,24.0,24,...,0,1000.00,0,0,0,0,833.33,0,0,757.14,0,680.00,0,0,7.70,733.33,0,1,55.8961,19284.1,-4.5,14.788,18.6,-0.065116,5.97,7.12,1639.70,118.73,0.026267,8.74,458999,13.29,63194.0,-0.079,0.017708,57746.0,8.3,211.83,39.62,30.98
2,30476,40.50,3,1960.0,0.0,84.0,9.946335e+06,139322,0.065409,0.225825,0,1548.0,3,0,13,0,0.0,0,2,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,36154,0.0,437.0,44,1.585306,0.580638,0.900408,11.749900,3.389848,10.195630,3.628293,68.0,59,...,0,485.71,0,0,6,2,655.56,0,3,633.33,0,608.33,0,4,13.59,668.97,0,11,55.8961,19284.1,-4.5,14.788,18.6,-0.065116,5.97,7.12,1639.70,118.73,0.026267,8.74,458999,13.29,63194.0,-0.079,0.017708,57746.0,8.3,211.83,39.62,30.98
3,30477,62.80,17,2016.0,1.0,104.0,2.149409e+07,7122,0.262459,0.017647,0,1046.0,0,0,0,2,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9553,3.0,281.0,45,7.931398,0.025446,0.466738,5.061917,2.701804,14.629440,10.284167,39.0,39,...,0,300.00,0,0,0,0,300.00,0,0,300.00,0,300.00,0,0,1.66,1560.00,0,0,55.8961,19284.1,-4.5,14.788,18.6,-0.065116,5.97,7.12,1639.70,118.73,0.026267,8.74,458999,13.29,63194.0,-0.079,0.017708,57746.0,8.3,211.83,39.62,30.98
4,30478,40.00,17,0.0,1.0,102.0,2.553630e+07,4001,0.496315,0.007122,0,1046.0,0,0,0,0,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17790,3.0,281.0,21,2.152792,0.427248,0.353642,16.784630,2.250137,19.149190,3.735666,24.0,24,...,0,1000.00,0,0,0,0,833.33,0,0,757.14,0,700.00,0,0,6.96,747.37,0,1,55.8961,19284.1,-4.5,14.788,18.6,-0.065116,5.97,7.12,1639.70,118.73,0.026267,8.74,458999,13.29,63194.0,-0.079,0.017708,57746.0,8.3,211.83,39.62,30.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

# Подготовка к ML. Тренировочная и валидационная выборки.

In [76]:
from sklearn.model_selection import train_test_split

In [129]:
target_train = train['price_doc']
X_train, X_val, y_train, y_val = train_test_split(train_final, target_train, test_size=0.2, random_state=1)

X_test = test_final

Напишем функции для отрисовки графиков цен. По оси абсцисс откладывается индекс строки выборки с данными, по оси ординат - цена недвижимости.

In [78]:
def val_table(num_samp): 
  return pd.DataFrame(data= {'id':X_val[:,:1].ravel(), 'Val': y_val}).set_index('id').sort_values(by=['id']).head(num_samp)

In [79]:
def predict_table(metod, num_samp):
  return pd.DataFrame(data= {'id':X_val[:,:1].ravel(), 'Predict': metod.predict(X_val)}).set_index('id').sort_values(by=['id']).head(num_samp)

# ML модели

## Решающее дерево

In [80]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import normalize
import pickle

In [81]:
dtree = DecisionTreeRegressor(criterion='squared_error', splitter='best', random_state=1, max_depth = 10)
dtree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=10, random_state=1)

In [82]:
print(
    '''
    - Точность на тестовой выборке: {}
    - Точность на валидационной выборке: {}
    '''.format(
        dtree.score(X_train, y_train),
        dtree.score(X_val, y_val)
        )
)


    - Точность на тестовой выборке: 0.792382441688182
    - Точность на валидационной выборке: 0.6119291042145669
    


Как видим, одно решающее дерево не способно дать хорошей точности на валидационной выборке.

## Линейная регрессия

In [83]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [84]:
%%time

linr = LinearRegression().fit(X_train, y_train)

CPU times: user 472 ms, sys: 34.6 ms, total: 507 ms
Wall time: 312 ms


In [85]:
print(
    '''
    - Точность на тестовой выборке: {}
    - Точность на валидационной выборке: {}
    '''.format(
        linr.score(X_train, y_train),
        linr.score(X_val, y_val)
        )
)


    - Точность на тестовой выборке: 0.31635325322847596
    - Точность на валидационной выборке: 0.3795412640411874
    


Результат крайне не удовлетворительный. Что интересно, на слабо обработанной выборке (все NaN заменены нулями, категориальные признаки удалены, параметры macro не учитывались) линейная регрессия показала значительно более лучший результат. Вероятно, сыграло роль большее коллличество признаков, даже не смотря на наличие в них множества сильно скоррелированных признаков.

## Случайный лес

In [86]:
from sklearn.ensemble import RandomForestRegressor

In [87]:
%%time

rf = RandomForestRegressor(n_estimators=15,criterion='squared_error', random_state=1, max_depth = 10)
rf.fit(X_train, y_train)

CPU times: user 11.6 s, sys: 9.79 ms, total: 11.6 s
Wall time: 11.6 s


In [88]:
print(
    '''
    - Точность на тестовой выборке: {}
    - Точность на валидационной выборке: {}
    '''.format(
        rf.score(X_train, y_train),
        rf.score(X_val, y_val)
        )
)


    - Точность на тестовой выборке: 0.8037237061118518
    - Точность на валидационной выборке: 0.7111997743440224
    


Ожидаемо лучше чем одно дерево. Создавать кол-во деревьев сильно больше 10 нет смысла, т.к. прирост времени вычисления значительный, а прироста точности почти не происходит. Увеличение глубины дерева улучшает результат на тренировочной выборке, но не дает прироста на валидационной выборке. Модель переобучается. 

## Градиентный бустинг

In [89]:
from sklearn.ensemble import GradientBoostingRegressor

In [90]:
# # Крос-валидация и подбор гиперпараметров

# %%time

# GB = GradientBoostingRegressor(loss='squared_error',random_state=1)
# GB_params = {'n_estimators': range(800,1000,100), 'max_depth': range(3,7,1)}
# GB_grid = GridSearchCV(GB, GB_params, cv=2, n_jobs=-1)
# GB_grid.fit(X_train, y_train)

# print(
#     '''
#     - Лучшие гиперпараметры: {}
#     - Точность: {}
#     '''.format(
#         GB_grid.best_params_,
#         GB_grid.best_score_)
# )

In [91]:
%%time

GB = GradientBoostingRegressor(loss='squared_error',random_state=1, n_estimators=500, max_depth=5)
GB.fit(X_train, y_train)

CPU times: user 4min 52s, sys: 125 ms, total: 4min 52s
Wall time: 4min 51s


In [92]:
print(
    '''
    - Точность на тестовой выборке: {}
    - Точность на валидационной выборке: {}
    '''.format(
        GB.score(X_train, y_train),
        GB.score(X_val, y_val)
        )
)


    - Точность на тестовой выборке: 0.9049008278278141
    - Точность на валидационной выборке: 0.7488273783312391
    


Градиентный бустинг ожидаемо показывает лучший результат. Один минус - скорость вычислений. 

# Поиск Price на тестовой выборке. Результат kaggle.

In [143]:
GB_Price = GB.predict(X_test)

In [140]:
rf_Price = rf.predict(X_test)

In [132]:
from pathlib import Path  

In [144]:
GB_out = pd.DataFrame({'id':X_test.id, 'price_doc':GB_Price}).set_index(['id'])

In [148]:
rf_out = pd.DataFrame({'id':X_test.id, 'price_doc':rf_Price}).set_index(['id'])

In [145]:
GB_filepath = Path('/content/drive/MyDrive/Colab Notebooks/ДЗ. Большое 3/GB_out.csv')  
rf_filepath = Path('/content/drive/MyDrive/Colab Notebooks/ДЗ. Большое 3/rf_out.csv')  

In [146]:
GB_out.to_csv(GB_filepath)  

In [149]:
rf_out.to_csv(rf_filepath)  

Неожиданый результат работы GB. Метод выдал несколько отрицательных ответов, решение не проходит проверку kaggle. Случайный лес выдает положительные ответы, результат kaggle - 0.33120. Примерно 2300-е место в рейтинге.

В качестве эксперимента в kaggle был загружено предсказание методом случайного леса для слабо очищенной выборки (все категориальные признаки удалены, все NaN заменены нулями, выборка macro не рассматривалась). Результат kaggle - 0.33304. Место в рейтинге - 2347. Метод градиентного бустинга также дал несколько отрицательных результатов. 

Таким образом, можно сделать вывод, что проведенный EDA дал некоторые результаты. Но не значительные. В рамках подготовки группового задания, будет проведена работа по усовершенствованию алгоритма EDA и выяснению причин отрицательных ответов на градиентном бустинге. 